In [1]:
import json
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os

In [2]:
path = '/Users/minhkhoa/Documents/used-cars-prices-prediction/data/sanxehot/'
df = pd.read_json(path + 'sanxehot_links.json').T
df.head()

,url,type
https://www.sanxehot.vn/mua-ban-xe/mercedes--benz-class-27548.html,https://www.sanxehot.vn/mua-ban-xe/mercedes--b...,Sedan
https://www.sanxehot.vn/mua-ban-xe/mercedes--benz-class-27547.html,https://www.sanxehot.vn/mua-ban-xe/mercedes--b...,Sedan
https://www.sanxehot.vn/mua-ban-xe/kia-new-sorento-22at-2014-26003.html,https://www.sanxehot.vn/mua-ban-xe/kia-new-sor...,SUV 7 chỗ
https://www.sanxehot.vn/mua-ban-xe/mercedes--benz-s400-30at-2016-26002.html,https://www.sanxehot.vn/mua-ban-xe/mercedes--b...,Sedan
https://www.sanxehot.vn/mua-ban-xe/lexus-rx350-35at-2011-26000.html,https://www.sanxehot.vn/mua-ban-xe/lexus-rx350...,SUV 5 chỗ


In [3]:
df.type.unique()

array(['Sedan', 'SUV 7 chỗ', 'SUV 5 chỗ', 'Hatchback', 'Bán tải',
       '7 chỗ đa dụng', '16 chỗ', 'Coupe', ''], dtype=object)

In [4]:
df.iloc[7845]

url     https://www.sanxehot.vn/mua-ban-xe/ford-fiesta...
type                                                Sedan
Name: https://www.sanxehot.vn/mua-ban-xe/ford-fiesta-16at-2011-25854.html, dtype: object

In [5]:
print(len(df))

7846


In [15]:
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.11; rv:46.0) "
                  "Gecko/20100101 Firefox/46.0",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.5",
    "Connection": "keep-alive",
    "Cache-Control": "max-age=0"
}

class SanXeHotCrawler:
    def __init__(self):
        self.df = pd.read_json(path + 'sanxehot_links.json').T

    def crawl(self):
        car_df = pd.DataFrame(columns=['name', 'brand', 'source_url', 'type', 'origin', 'km_driven',
                                       'external_color','seats', 'engine_capacity', 'fuels',
                                       'transmission', 'wheel_drive', 'price', 'year'])

        num = 0
        for i in range(0, 101):
            url = df.iloc[i]['url']
            type = df.iloc[i]['type']
            req = requests.get(url, headers=HEADERS).text
            soup = BeautifulSoup(req, 'lxml')
            body_page = soup.find('table', class_='info').tbody
            car_feature = body_page.find_all('td')

            gia = car_feature[0].text
            gia = gia.replace(',', '')
            if 'triệu' in gia:
                price = gia.replace(' triệu', '000000')
            elif 'tỷ' in gia:
                price = gia.replace(' tỷ', '000000')

            name = car_feature[2].text
            year = car_feature[4].text
            # if len(nam) > 1:
            #     year = nam[-1]
            # else:
            #     year = nam[0]

            km_driven = car_feature[6].text.split()[0]
            km_driven = km_driven.replace('.', '')

            dong_co = car_feature[8].text.split()
            if len(dong_co) == 2:
                fuels = dong_co[0]
                engine_capacity = dong_co[1].split('L')[0]
            else:
                fuels = dong_co[0]
                engine_capacity = None

            transmission = car_feature[10].text
            origin = car_feature[12].text
            external_color = car_feature[14].text

            car_df = car_df.append(pd.Series({'name': name, 'brand': None, 'source_url': url, 'type': type,
                                              'origin': origin, 'km_driven': km_driven, 'external_color': external_color,
                                              'seats': None, 'engine_capacity': engine_capacity, 'fuels': fuels,
                                              'transmission': transmission, 'wheel_drive': None,
                                              'price': price, 'year': year}), ignore_index=True)
            num += 1
            print(num)

        return car_df

In [16]:
cars = SanXeHotCrawler().crawl()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101


NameError: name 'cars' is not defined

In [19]:
cars.to_csv(path + 'test_df1.csv')